# HW07 – Кластеризация на синтетических данных


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score

import warnings
warnings.filterwarnings('ignore')

# Настройка отображения графиков
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Константы
RANDOM_STATE = 42
DATA_PATH = "./data/"
ARTIFACTS_PATH = "./artifacts/"
FIGURES_PATH = "./artifacts/figures/"


# 1. ФУНКЦИИ ДЛЯ АНАЛИЗА


In [ ]:
def load_dataset(filename):
    """Загрузка датасета из data/"""
    path = os.path.join(DATA_PATH, filename)
    df = pd.read_csv(path)
    print(f"Загружен {filename}: {df.shape[0]} строк, {df.shape[1]} столбцов")
    return df

def basic_eda(df, dataset_name):
    """Базовый анализ датасета"""
    print(f"\n{'='*60}")
    print(f"АНАЛИЗ ДАТАСЕТА: {dataset_name}")
    print(f"{'='*60}")
    
    print("\nПервые 5 строк:")
    display(df.head())
    
    print("\nИнформация о типах данных:")
    df.info()
    
    print("\nБазовые статистики:")
    display(df.describe())
    
    print("\nПропуски данных:")
    missing = df.isnull().sum()
    print(missing[missing > 0] if missing.any() else "Пропусков нет")
    
    print(f"\nУникальные значения в sample_id: {df['sample_id'].nunique()}")
    
    return df

def preprocess_data(df, dataset_name):
    """
    Препроцессинг данных
    Возвращает:
    - X_scaled: масштабированные признаки
    - X_raw: исходные признаки (без sample_id)
    - sample_ids
    """
    # Сохраняем sample_id отдельно
    sample_ids = df['sample_id'].copy()
    
    # Признаки (все кроме sample_id)
    X_raw = df.drop('sample_id', axis=1)
    
    # Проверяем типы признаков
    numeric_cols = X_raw.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X_raw.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"\nПрепроцессинг {dataset_name}:")
    print(f"  Числовые признаки: {len(numeric_cols)}")
    print(f"  Категориальные признаки: {len(categorical_cols)}")
    
    # Для HW07 все признаки числовые, но оставим общий случай
    if len(categorical_cols) > 0:
        print("  ВНИМАНИЕ: Есть категориальные признаки! Требуется кодирование.")
        # Здесь можно добавить OneHotEncoder
        from sklearn.preprocessing import OneHotEncoder
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_cols),
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
            ])
    else:
        # Только масштабирование для числовых признаков
        preprocessor = StandardScaler()
    
    # Применяем препроцессинг
    X_scaled = preprocessor.fit_transform(X_raw)
    
    # Если это был ColumnTransformer, преобразуем в массив
    if hasattr(X_scaled, 'toarray'):
        X_scaled = X_scaled.toarray()
    
    print(f"  Размер после препроцессинга: {X_scaled.shape}")
    
    return X_scaled, X_raw, sample_ids

def evaluate_clustering(X, labels, dataset_name, model_name):
    """Оценка качества кластеризации"""
    # Исключаем шумные точки для DBSCAN (-1)
    if model_name == 'DBSCAN':
        non_noise_mask = labels != -1
        if non_noise_mask.sum() > 1:  # Нужно хотя бы 2 точки для метрик
            X_eval = X[non_noise_mask]
            labels_eval = labels[non_noise_mask]
            noise_ratio = (labels == -1).mean()
        else:
            print("  ВНИМАНИЕ: Все точки помечены как шум!")
            return {
                'silhouette': np.nan,
                'davies_bouldin': np.nan,
                'calinski_harabasz': np.nan,
                'noise_ratio': 1.0
            }
    else:
        X_eval = X
        labels_eval = labels
        noise_ratio = 0.0
    
    # Считаем метрики
    n_clusters = len(np.unique(labels_eval))
    
    if n_clusters > 1:
        silhouette = silhouette_score(X_eval, labels_eval)
        db = davies_bouldin_score(X_eval, labels_eval)
        ch = calinski_harabasz_score(X_eval, labels_eval)
    else:
        silhouette = np.nan
        db = np.nan
        ch = np.nan
    
    metrics = {
        'silhouette': float(silhouette) if not np.isnan(silhouette) else None,
        'davies_bouldin': float(db) if not np.isnan(db) else None,
        'calinski_harabasz': float(ch) if not np.isnan(ch) else None,
        'n_clusters': int(n_clusters),
        'noise_ratio': float(noise_ratio) if model_name == 'DBSCAN' else 0.0
    }
    
    print(f"  Метрики для {model_name}:")
    print(f"    Кластеров: {metrics['n_clusters']}")
    if model_name == 'DBSCAN':
        print(f"    Доля шума: {metrics['noise_ratio']:.2%}")
    print(f"    Silhouette: {metrics['silhouette']:.3f}" if metrics['silhouette'] else "    Silhouette: N/A")
    print(f"    Davies-Bouldin: {metrics['davies_bouldin']:.3f}" if metrics['davies_bouldin'] else "    Davies-Bouldin: N/A")
    print(f"    Calinski-Harabasz: {metrics['calinski_harabasz']:.3f}" if metrics['calinski_harabasz'] else "    Calinski-Harabasz: N/A")
    
    return metrics

def visualize_clusters_pca(X, labels, dataset_name, model_name, params, save=False):
    """Визуализация кластеров в 2D PCA"""
    # PCA для визуализации
    pca = PCA(n_components=2, random_state=RANDOM_STATE)
    X_pca = pca.fit_transform(X)
    
    # Создаем DataFrame для удобства
    df_viz = pd.DataFrame({
        'PC1': X_pca[:, 0],
        'PC2': X_pca[:, 1],
        'cluster': labels
    })
    
    # Разделяем шум и кластеры для DBSCAN
    if model_name == 'DBSCAN':
        noise_mask = df_viz['cluster'] == -1
        df_clusters = df_viz[~noise_mask]
        df_noise = df_viz[noise_mask]
    else:
        df_clusters = df_viz
        df_noise = None
    
    # Визуализация
    plt.figure(figsize=(10, 8))
    
    # Рисуем кластеры
    scatter = plt.scatter(df_clusters['PC1'], df_clusters['PC2'], 
                          c=df_clusters['cluster'], cmap='tab20', 
                          s=50, alpha=0.8, edgecolors='k', linewidth=0.5)
    
    # Рисуем шум отдельно (для DBSCAN)
    if df_noise is not None and len(df_noise) > 0:
        plt.scatter(df_noise['PC1'], df_noise['PC2'], 
                   c='gray', s=30, alpha=0.3, marker='x', 
                   label='Noise (cluster -1)')
    
    # Настройки графика
    plt.colorbar(scatter, label='Cluster')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    
    title = f'{dataset_name} - {model_name}'
    if params:
        title += f'\n{params}'
    plt.title(title)
    
    if df_noise is not None and len(df_noise) > 0:
        plt.legend()
    
    plt.tight_layout()
    
    if save:
        filename = f"{dataset_name}_{model_name}_pca.png".replace(' ', '_').lower()
        filepath = os.path.join(FIGURES_PATH, filename)
        plt.savefig(filepath, dpi=150, bbox_inches='tight')
        print(f"  График сохранен: {filepath}")
    
    plt.show()
    
    # Возвращаем объясненную дисперсию
    return pca.explained_variance_ratio_.sum()

def plot_silhouette_vs_k(X, dataset_name, k_range=(2, 20), save=False):
    """График silhouette vs k для KMeans"""
    silhouette_scores = []
    
    for k in range(k_range[0], k_range[1]+1):
        kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
        labels = kmeans.fit_predict(X)
        
        if len(np.unique(labels)) > 1:
            score = silhouette_score(X, labels)
        else:
            score = 0
        silhouette_scores.append(score)
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(k_range[0], k_range[1]+1), silhouette_scores, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title(f'{dataset_name} - Silhouette Score vs k for KMeans')
    plt.grid(True, alpha=0.3)
    
    # Отмечаем лучший k
    best_k = range(k_range[0], k_range[1]+1)[np.argmax(silhouette_scores)]
    plt.axvline(x=best_k, color='r', linestyle='--', alpha=0.7, 
                label=f'Best k = {best_k}')
    plt.legend()
    
    plt.tight_layout()
    
    if save:
        filename = f"{dataset_name}_silhouette_vs_k.png".replace(' ', '_').lower()
        filepath = os.path.join(FIGURES_PATH, filename)
        plt.savefig(filepath, dpi=150, bbox_inches='tight')
    
    plt.show()
    
    return best_k, max(silhouette_scores)

def save_results(dataset_name, model_name, params, metrics, labels, sample_ids):
    """Сохранение результатов"""
    # Сохраняем метки кластеров
    results_df = pd.DataFrame({
        'sample_id': sample_ids,
        'cluster_label': labels
    })
    
    filename = f"labels_{dataset_name}_{model_name}.csv".replace(' ', '_').lower()
    filepath = os.path.join(ARTIFACTS_PATH, 'labels', filename)
    
    # Создаем директорию если нужно
    Path(os.path.join(ARTIFACTS_PATH, 'labels')).mkdir(exist_ok=True)
    
    results_df.to_csv(filepath, index=False)
    print(f"  Метки кластеров сохранены: {filepath}")
    
    return results_df

# 2. АНАЛИЗ ДАТАСЕТА 01

In [ ]:
print("="*80)
print("ДАТАСЕТ 01: Числовые признаки в разных шкалах + шумовые признаки")
print("="*80)

# Загрузка данных
df1 = load_dataset("S07-hw-dataset-01.csv")
df1 = basic_eda(df1, "Dataset 01")

# Препроцессинг
X1_scaled, X1_raw, sample_ids1 = preprocess_data(df1, "Dataset 01")

# 2.1 KMeans для Dataset 01


In [ ]:
print("\n" + "="*60)
print("KMEANS для Dataset 01")
print("="*60)

# Подбор оптимального k
best_k1, best_sil1 = plot_silhouette_vs_k(X1_scaled, "Dataset 01", k_range=(2, 15), save=True)

# Лучшая модель KMeans
kmeans1 = KMeans(n_clusters=best_k1, random_state=RANDOM_STATE, n_init=10)
labels_kmeans1 = kmeans1.fit_predict(X1_scaled)

# Оценка
metrics_kmeans1 = evaluate_clustering(X1_scaled, labels_kmeans1, "Dataset 01", "KMeans")

# Визуализация
var_exp1_kmeans = visualize_clusters_pca(X1_scaled, labels_kmeans1, 
                                        "Dataset 01", "KMeans", 
                                        f"k={best_k1}", save=True)

# Сохранение результатов
save_results("dataset01", "kmeans", 
            {"n_clusters": best_k1, "random_state": RANDOM_STATE},
            metrics_kmeans1, labels_kmeans1, sample_ids1)

# 2.2 DBSCAN для Dataset 01


In [ ]:
print("\n" + "="*60)
print("DBSCAN для Dataset 01")
print("="*60)

# Подбор параметров DBSCAN
eps_values = [0.3, 0.5, 0.7, 1.0, 1.5, 2.0]
min_samples_values = [3, 5, 7, 10]

best_eps1 = None
best_min_samples1 = None
best_silhouette1_db = -1
best_labels1_db = None

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X1_scaled)
        
        # Исключаем шум для вычисления метрик
        non_noise_mask = labels != -1
        if non_noise_mask.sum() > 1 and len(np.unique(labels[non_noise_mask])) > 1:
            silhouette = silhouette_score(X1_scaled[non_noise_mask], labels[non_noise_mask])
        else:
            silhouette = -1
        
        if silhouette > best_silhouette1_db:
            best_silhouette1_db = silhouette
            best_eps1 = eps
            best_min_samples1 = min_samples
            best_labels1_db = labels

print(f"Лучшие параметры DBSCAN: eps={best_eps1}, min_samples={best_min_samples1}")
print(f"Лучший silhouette: {best_silhouette1_db:.3f}")

# Оценка лучшей модели DBSCAN
dbscan1 = DBSCAN(eps=best_eps1, min_samples=best_min_samples1)
labels_dbscan1 = dbscan1.fit_predict(X1_scaled)

metrics_dbscan1 = evaluate_clustering(X1_scaled, labels_dbscan1, "Dataset 01", "DBSCAN")

# Визуализация
var_exp1_dbscan = visualize_clusters_pca(X1_scaled, labels_dbscan1, 
                                        "Dataset 01", "DBSCAN", 
                                        f"eps={best_eps1}, min_samples={best_min_samples1}", 
                                        save=True)

# Сохранение результатов
save_results("dataset01", "dbscan", 
            {"eps": best_eps1, "min_samples": best_min_samples1},
            metrics_dbscan1, labels_dbscan1, sample_ids1)

# 3. АНАЛИЗ ДАТАСЕТА 02


In [ ]:
print("\n" + "="*80)
print("ДАТАСЕТ 02: Нелинейная структура + выбросы + шумовой признак")
print("="*80)

# Загрузка данных
df2 = load_dataset("S07-hw-dataset-02.csv")
df2 = basic_eda(df2, "Dataset 02")

# Препроцессинг
X2_scaled, X2_raw, sample_ids2 = preprocess_data(df2, "Dataset 02")

# 3.1 KMeans для Dataset 02


In [ ]:
print("\n" + "="*60)
print("KMEANS для Dataset 02")
print("="*60)

# Подбор оптимального k
best_k2, best_sil2 = plot_silhouette_vs_k(X2_scaled, "Dataset 02", k_range=(2, 15), save=True)

# Лучшая модель KMeans
kmeans2 = KMeans(n_clusters=best_k2, random_state=RANDOM_STATE, n_init=10)
labels_kmeans2 = kmeans2.fit_predict(X2_scaled)

# Оценка
metrics_kmeans2 = evaluate_clustering(X2_scaled, labels_kmeans2, "Dataset 02", "KMeans")

# Визуализация
var_exp2_kmeans = visualize_clusters_pca(X2_scaled, labels_kmeans2, 
                                        "Dataset 02", "KMeans", 
                                        f"k={best_k2}", save=True)

# Сохранение результатов
save_results("dataset02", "kmeans", 
            {"n_clusters": best_k2, "random_state": RANDOM_STATE},
            metrics_kmeans2, labels_kmeans2, sample_ids2)

# 3.2 Agglomerative Clustering для Dataset 02


In [ ]:
print("\n" + "="*60)
print("AGGLOMERATIVE CLUSTERING для Dataset 02")
print("="*60)

# Тестируем разные linkage
linkage_methods = ['ward', 'complete', 'average', 'single']
best_linkage2 = None
best_silhouette2_agg = -1
best_labels2_agg = None

for linkage in linkage_methods:
    # Используем тот же k, что и у KMeans для сравнения
    agg = AgglomerativeClustering(n_clusters=best_k2, linkage=linkage)
    labels = agg.fit_predict(X2_scaled)
    
    if len(np.unique(labels)) > 1:
        silhouette = silhouette_score(X2_scaled, labels)
    else:
        silhouette = -1
    
    if silhouette > best_silhouette2_agg:
        best_silhouette2_agg = silhouette
        best_linkage2 = linkage
        best_labels2_agg = labels

print(f"Лучший linkage: {best_linkage2}")
print(f"Лучший silhouette: {best_silhouette2_agg:.3f}")

# Лучшая модель Agglomerative
agg2 = AgglomerativeClustering(n_clusters=best_k2, linkage=best_linkage2)
labels_agg2 = agg2.fit_predict(X2_scaled)

metrics_agg2 = evaluate_clustering(X2_scaled, labels_agg2, "Dataset 02", "Agglomerative")

# Визуализация
var_exp2_agg = visualize_clusters_pca(X2_scaled, labels_agg2, 
                                     "Dataset 02", "Agglomerative", 
                                     f"k={best_k2}, linkage={best_linkage2}", 
                                     save=True)

# Сохранение результатов
save_results("dataset02", "agglomerative", 
            {"n_clusters": best_k2, "linkage": best_linkage2},
            metrics_agg2, labels_agg2, sample_ids2)

# 4. АНАЛИЗ ДАТАСЕТА 03


In [ ]:

print("\n" + "="*80)
print("ДАТАСЕТ 03: Кластеры разной плотности + фоновый шум")
print("="*80)

# Загрузка данных
df3 = load_dataset("S07-hw-dataset-03.csv")
df3 = basic_eda(df3, "Dataset 03")

# Препроцессинг
X3_scaled, X3_raw, sample_ids3 = preprocess_data(df3, "Dataset 03")

# 4.1 KMeans для Dataset 03


In [ ]:
print("\n" + "="*60)
print("KMEANS для Dataset 03")
print("="*60)

# Подбор оптимального k
best_k3, best_sil3 = plot_silhouette_vs_k(X3_scaled, "Dataset 03", k_range=(2, 15), save=True)

# Лучшая модель KMeans
kmeans3 = KMeans(n_clusters=best_k3, random_state=RANDOM_STATE, n_init=10)
labels_kmeans3 = kmeans3.fit_predict(X3_scaled)

# Оценка
metrics_kmeans3 = evaluate_clustering(X3_scaled, labels_kmeans3, "Dataset 03", "KMeans")

# Визуализация
var_exp3_kmeans = visualize_clusters_pca(X3_scaled, labels_kmeans3, 
                                        "Dataset 03", "KMeans", 
                                        f"k={best_k3}", save=True)

# Сохранение результатов
save_results("dataset03", "kmeans", 
            {"n_clusters": best_k3, "random_state": RANDOM_STATE},
            metrics_kmeans3, labels_kmeans3, sample_ids3)

# 4.2 DBSCAN для Dataset 03 (хорошо для разной плотности)


In [ ]:
print("\n" + "="*60)
print("DBSCAN для Dataset 03")
print("="*60)

# Для датасета с разной плотностью DBSCAN может быть лучше
# Используем k-distance plot для выбора eps
from sklearn.neighbors import NearestNeighbors

# k-distance plot
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(X3_scaled)
distances, indices = neighbors_fit.kneighbors(X3_scaled)

# Сортируем расстояния до 5-го соседа
k_distance = np.sort(distances[:, 4])

plt.figure(figsize=(10, 6))
plt.plot(range(len(k_distance)), k_distance)
plt.xlabel('Points sorted by distance to 5th nearest neighbor')
plt.ylabel('5th nearest neighbor distance')
plt.title('Dataset 03 - k-distance plot for DBSCAN (k=5)')
plt.grid(True, alpha=0.3)

# Отмечаем возможные значения eps
for eps in [0.5, 1.0, 1.5, 2.0]:
    plt.axhline(y=eps, color='r', linestyle='--', alpha=0.5)
    
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_PATH, "dataset03_k_distance_plot.png"), dpi=150)
plt.show()

# Подбор параметров DBSCAN
eps_values = [0.8, 1.0, 1.2, 1.5, 2.0]
min_samples_values = [3, 5, 7]

best_eps3 = None
best_min_samples3 = None
best_silhouette3_db = -1
best_labels3_db = None

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X3_scaled)
        
        # Исключаем шум для вычисления метрик
        non_noise_mask = labels != -1
        if non_noise_mask.sum() > 1 and len(np.unique(labels[non_noise_mask])) > 1:
            silhouette = silhouette_score(X3_scaled[non_noise_mask], labels[non_noise_mask])
        else:
            silhouette = -1
        
        if silhouette > best_silhouette3_db:
            best_silhouette3_db = silhouette
            best_eps3 = eps
            best_min_samples3 = min_samples
            best_labels3_db = labels

print(f"Лучшие параметры DBSCAN: eps={best_eps3}, min_samples={best_min_samples3}")
print(f"Лучший silhouette: {best_silhouette3_db:.3f}")

# Лучшая модель DBSCAN
dbscan3 = DBSCAN(eps=best_eps3, min_samples=best_min_samples3)
labels_dbscan3 = dbscan3.fit_predict(X3_scaled)

metrics_dbscan3 = evaluate_clustering(X3_scaled, labels_dbscan3, "Dataset 03", "DBSCAN")

# Визуализация
var_exp3_dbscan = visualize_clusters_pca(X3_scaled, labels_dbscan3, 
                                        "Dataset 03", "DBSCAN", 
                                        f"eps={best_eps3}, min_samples={best_min_samples3}", 
                                        save=True)

# Сохранение результатов
save_results("dataset03", "dbscan", 
            {"eps": best_eps3, "min_samples": best_min_samples3},
            metrics_dbscan3, labels_dbscan3, sample_ids3)

# 5. ПРОВЕРКА УСТОЙЧИВОСТИ (для Dataset 01)


In [ ]:
print("\n" + "="*80)
print("ПРОВЕРКА УСТОЙЧИВОСТИ KMeans для Dataset 01")
print("="*80)

# Проверяем устойчивость KMeans с разными random_state
n_runs = 5
ari_scores = []

# Первый запуск как базовый
kmeans_base = KMeans(n_clusters=best_k1, random_state=RANDOM_STATE, n_init=10)
labels_base = kmeans_base.fit_predict(X1_scaled)

for i in range(n_runs):
    kmeans_temp = KMeans(n_clusters=best_k1, random_state=i*100, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X1_scaled)
    
    # Вычисляем ARI между базовой и текущей кластеризацией
    ari = adjusted_rand_score(labels_base, labels_temp)
    ari_scores.append(ari)
    
    print(f"Запуск {i+1} (random_state={i*100}): ARI = {ari:.4f}")

print(f"\nСредний ARI: {np.mean(ari_scores):.4f}")
print(f"Стандартное отклонение ARI: {np.std(ari_scores):.4f}")

if np.mean(ari_scores) > 0.9:
    print("Кластеризация УСТОЙЧИВА (высокое согласие между запусками)")
elif np.mean(ari_scores) > 0.7:
    print("Кластеризация УМЕРЕННО УСТОЙЧИВА")
else:
    print("Кластеризация НЕУСТОЙЧИВА")

# Визуализация результатов разных запусков
fig, axes = plt.subplots(1, n_runs, figsize=(20, 4))

for i, ax in enumerate(axes):
    kmeans_temp = KMeans(n_clusters=best_k1, random_state=i*100, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X1_scaled)
    
    # PCA для визуализации
    pca = PCA(n_components=2, random_state=RANDOM_STATE)
    X_pca = pca.fit_transform(X1_scaled)
    
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=labels_temp, cmap='tab20', s=30, alpha=0.8)
    ax.set_title(f'Run {i+1}, seed={i*100}\nARI={ari_scores[i]:.3f}')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')

plt.suptitle(f'Устойчивость KMeans для Dataset 01 (k={best_k1})', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_PATH, "dataset01_stability_kmeans.png"), dpi=150)
plt.show()

# 6. СВОДКА РЕЗУЛЬТАТОВ


In [ ]:
print("\n" + "="*80)
print("СВОДКА РЕЗУЛЬТАТОВ ПО ВСЕМ ДАТАСЕТАМ")
print("="*80)

# Собираем все метрики
summary = {
    'dataset01': {
        'kmeans': {
            'params': {'n_clusters': best_k1, 'random_state': RANDOM_STATE},
            'metrics': metrics_kmeans1
        },
        'dbscan': {
            'params': {'eps': best_eps1, 'min_samples': best_min_samples1},
            'metrics': metrics_dbscan1
        }
    },
    'dataset02': {
        'kmeans': {
            'params': {'n_clusters': best_k2, 'random_state': RANDOM_STATE},
            'metrics': metrics_kmeans2
        },
        'agglomerative': {
            'params': {'n_clusters': best_k2, 'linkage': best_linkage2},
            'metrics': metrics_agg2
        }
    },
    'dataset03': {
        'kmeans': {
            'params': {'n_clusters': best_k3, 'random_state': RANDOM_STATE},
            'metrics': metrics_kmeans3
        },
        'dbscan': {
            'params': {'eps': best_eps3, 'min_samples': best_min_samples3},
            'metrics': metrics_dbscan3
        }
    },
    'stability_check': {
        'dataset': 'dataset01',
        'method': 'kmeans',
        'n_runs': n_runs,
        'mean_ari': float(np.mean(ari_scores)),
        'std_ari': float(np.std(ari_scores))
    }
}

# Сохраняем сводку в JSON
summary_path = os.path.join(ARTIFACTS_PATH, 'metrics_summary.json')
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"Сводка метрик сохранена: {summary_path}")

# Сохраняем лучшие конфигурации
best_configs = {}
for dataset, models in summary.items():
    if dataset != 'stability_check':
        # Выбираем лучший метод по silhouette
        best_model = None
        best_score = -1
        
        for model_name, data in models.items():
            silhouette = data['metrics'].get('silhouette', -1)
            if silhouette is not None and silhouette > best_score:
                best_score = silhouette
                best_model = model_name
        
        if best_model:
            best_configs[dataset] = {
                'best_model': best_model,
                'params': models[best_model]['params'],
                'silhouette': best_score
            }

best_configs_path = os.path.join(ARTIFACTS_PATH, 'best_configs.json')
with open(best_configs_path, 'w') as f:
    json.dump(best_configs, f, indent=2)
print(f"Лучшие конфигурации сохранены: {best_configs_path}")

# Выводим таблицу сравнения
print("\n" + "="*80)
print("ТАБЛИЦА СРАВНЕНИЯ МЕТОДОВ")
print("="*80)

for dataset in ['dataset01', 'dataset02', 'dataset03']:
    print(f"\n{dataset.upper()}:")
    print("-"*40)
    
    if dataset in summary:
        for model_name, data in summary[dataset].items():
            metrics = data['metrics']
            print(f"  {model_name.upper():15} | ", end="")
            print(f"Clusters: {metrics.get('n_clusters', 'N/A'):3} | ", end="")
            
            if model_name == 'dbscan':
                print(f"Noise: {metrics.get('noise_ratio', 0):.1%} | ", end="")
            
            silhouette = metrics.get('silhouette')
            if silhouette is not None:
                print(f"Silhouette: {silhouette:.3f}")
            else:
                print(f"Silhouette: N/A")

# 7. ВЫВОДЫ ПО ДАТАСЕТАМ

1. DATASET 01 (числовые признаки в разных шкалах):
   - Масштабирование было критически важно
   - KMeans показал хорошие результаты при k=2 (silhouette=0.522)
   - DBSCAN с параметрами eps=0.5, min_samples=5 дал аналогичный результат
   - Оба метода выделили 2 кластера, результаты устойчивы (ARI=1.0)
   - Лучший метод: KMeans (проще интерпретировать, нет шума)

2. DATASET 02 (нелинейная структура + шум):
   - KMeans показал низкое качество (silhouette=0.307) из-за нелинейной формы
   - Agglomerative с linkage='single' значительно лучше (silhouette=0.521)
   - DB index 0.342 vs 1.323 подтверждает превосходство иерархической кластеризации
   - Single linkage лучше всего справился с нелинейными кластерами
   - Лучший метод: AgglomerativeClustering с linkage='single'

3. DATASET 03 (разная плотность + шум):
   - KMeans выделил 3 кластера с silhouette=0.316
   - DBSCAN с eps=0.8, min_samples=3 показал лучше (silhouette=0.373)
   - DBSCAN корректно выделил шум (0.15% точек)
   - DB index 0.551 vs 1.158 показывает лучшее разделение DBSCAN
   - Лучший метод: DBSCAN (лучше адаптируется к разной плотности)

4. ОБЩИЕ НАБЛЮДЕНИЯ:
   - Масштабирование обязательно для distance-based методов
   - Silhouette score хорош для подбора k в KMeans
   - DBSCAN чувствителен к параметрам eps и min_samples
   - k-distance plot полезен для выбора eps в DBSCAN
   - Для нелинейных кластеров иерархическая кластеризация лучше KMeans
   - Для кластеров разной плотности DBSCAN предпочтительнее
   - Визуализация в PCA помогает интерпретировать результаты